In [1]:
import warnings

warnings.filterwarnings('ignore')

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [19]:
titleX = []
spX = []
mrpX = []
discountX = []
linkX = []
quantityX = []
categoryX = []
ratingX = []
rating_countX = []
review_countX = []

for page in range(1, 2):
     # target URL to scrap
    url = f"https://www.spencers.in/groceries-staples/dal-beans.html"
    
    # headers
    headers = {
        'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
        }
    # send request to download the data
    response = requests.request("GET", url, headers=headers)

    # parse the downloaded data
    data = BeautifulSoup(response.text, 'html.parser')
    
    dal_data = data.find_all('li', attrs={'class', 'item product product-item col-md-3 col-sm-3 col-xs-6'})
    
    print('Number of products : ', len(dal_data))
    
    for dal in dal_data:
        dal_name_qty = dal.find('h2', attrs={'class': 'product-name'})
        dal_name_qty_diff = re.findall('\d*\D+',dal_name_qty.text.strip())
        titleX.append(dal_name_qty_diff[0])
        print(dal_name_qty_diff[0])
        if len(dal_name_qty_diff) > 1:
            quantityX.append(dal_name_qty_diff[1])
        else:
            quantityX.append('N')
        
        price_sec = dal.find_all('span', attrs={'class': 'price'})
        actual_price = price_sec[0]
        if len(price_sec) > 1:
            offer_price = price_sec[1]
            act = float(price_sec[0].text.replace("₹", ""))
            off = float(price_sec[1].text.replace("₹", ""))
            discount = round((((act - off) / act) * 100), 2)
#             print(discount)
        else:
            offer_price = price_sec[0]
            discount = 0
            
        mrpX.append(actual_price.text)
        spX.append(offer_price.text)
        discountX.append(discount)
        
        product_link = dal.find('a', attrs={'class': 'product-item-link'})
        linkX.append(product_link['href'])
        
        # Handling unavailable data from the website
        categoryX.append('dal-beans')
        ratingX.append(99)
        rating_countX.append(99)
        review_countX.append(99)

Number of products :  12
Tata Sampann Chawli - Lobia 
Organic Tattva Chana Dal 
Spencer's Smart Choice Navratna Dal 
Spencer's Smart Choice Regular Moong Dal Dhuli 
Spencer's Smart Choice Navratna Dal 
Spencer's Smart Choice Regular Moong Dal Dhuli 
Premium Moong Dal 
Premium Moong Dal 
Spencer's Smart Choice Chana Dal 
Spencer's Smart Choice Chana Dal 
Premium Arhar Dal 
Premium Arhar Dal 


In [20]:
df = pd.DataFrame({'Title': titleX, 'Selling Price': spX, 'MRP': mrpX, 'Product Link': linkX, 'Quantity': quantityX,
                 'Discount': discountX, 'Rating': ratingX, 'Category': categoryX, 'Rating Count': rating_countX,
                 'Review Count': review_countX})

df.head()

,Title,Selling Price,MRP,Product Link,Quantity,Discount,Rating,Category,Rating Count,Review Count
0,Tata Sampann Chawli - Lobia,₹66,₹66,https://www.spencers.in/daily-essentials/tata-...,500g,0.00,99,dal-beans,99,99
1,Organic Tattva Chana Dal,₹95,₹95,https://www.spencers.in/daily-essentials/organ...,500g,0.00,99,dal-beans,99,99
2,Spencer's Smart Choice Navratna Dal,₹120,₹215,https://www.spencers.in/spencers-products/spen...,1kg,44.19,99,dal-beans,99,99
3,Spencer's Smart Choice Regular Moong Dal Dhuli,₹139,₹149,https://www.spencers.in/spencers-products/spen...,1kg,6.71,99,dal-beans,99,99
4,Spencer's Smart Choice Navratna Dal,₹60,₹107.50,https://www.spencers.in/spencers-products/spen...,500g,44.19,99,dal-beans,99,99


In [21]:
df.to_csv('spencers_dal.csv', index=False)